In [ ]:
!pip install tweepy
import tweepy
import datetime
import pytz
!pip install stop_words --use-pep517
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.util import ngrams
from stop_words import get_stop_words
import re
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk import bigrams

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abhit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# For a User Input on stock

In [ ]:
# Set up the API with your Twitter developer account's credentials
consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
stock_ticker = str(input('Enter the stock code'))
stock_ticker_full = str(input('Enter the stock name'))
stock_hashtag = '#' + stock_ticker

Enter the stock code ZOMATO
Enter the stock name Zomato


In [ ]:
data = []
usernames = ["wsj", "Reuters", "business","CNBC","WSJmarkets","livemint",
             "moneycontrolcom","NDTVProfit","bsindia","FinancialTimes",
             "ETMarkets","bqprime","Investopedia","BloombergTV","FinancialXpress",
             "NSEIndia","TOIBusiness","IIFL_Live","elearnmarkets","mystockedge"]

for username in usernames:
    tweets = api.user_timeline(screen_name=username, count=100)
    for tweet in tweets:
        if stock_ticker in tweet.text or stock_ticker_full in tweet.text:
            data.append(tweet.text)
            
tweets = api.search_tweets(q=stock_hashtag, count=100)
for tweet in tweets:
    data.append(tweet.text)

In [ ]:
stop_words = list(stopwords.words('english')) + list(get_stop_words('en'))
pattern = '[0-9]'
cleaned_data = []

def generate_clean_ngrams(tweet_text, n):

    tweet_text = tweet_text.lower()
    tweet_text = re.sub(r'[^a-zA-Z0-9\s]', ' ', tweet_text)
    tweet_text = re.sub(pattern, 'http', tweet_text)
    
    tokens = [token for token in tweet_text.split(' ') 
              if token != '' 
              and token not in stop_words 
              and 'http' not in token]
    
    output = list(ngrams(tokens, n))
    
    return output

for d in data:
    cleaned_data.append(generate_clean_ngrams(d, 2))

In [ ]:
analyzer = SentimentIntensityAnalyzer()
score = []

for data in cleaned_data:
    for d in data:
        bigram_text = " ".join(d)
        scores = analyzer.polarity_scores(bigram_text)
        score.append(scores['compound'])

In [ ]:
print(f'Sentiment Score for the stock {stock_ticker}: {sum(score)/len(score)}')

Sentiment Score for the stock ZOMATO: 0.008237021716649428


# For NSE top 500

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
nseList= pd.read_csv('C:/Users/abhit/Desktop/COLLEGE/SEM X/SEMX_Git/CC/ind_nifty500list.csv')
nseList.head()

,Company Name,Symbol
0,3M India Ltd.,3MINDIA
1,ABB India Ltd.,ABB
2,ACC Ltd.,ACC
3,AIA Engineering Ltd.,AIAENG
4,APL Apollo Tubes Ltd.,APLAPOLLO


In [ ]:
# Set up the API with your Twitter developer account's credentials
consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
nseList['Company Name'] = nseList['Company Name'].str.replace('.','')
nseList['Company Name'] = nseList['Company Name'].str.replace('Ltd','')

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_26204\571671230.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  nseList['Company Name'] = nseList['Company Name'].str.replace('.','')


In [ ]:
data = []
usernames = ["wsj", "Reuters", "business","CNBC","WSJmarkets","livemint",
             "moneycontrolcom","NDTVProfit","bsindia","FinancialTimes",
             "ETMarkets","bqprime","Investopedia","BloombergTV","FinancialXpress",
             "NSEIndia","TOIBusiness","IIFL_Live","elearnmarkets","mystockedge"]

In [ ]:
nseList['Sentiment'] = 0
for index in nseList.index:
    data = []
    stock_ticker = nseList['Symbol'][index]
    stock_ticker_full = nseList['Company Name'][index]
    stock_hashtag = '#' + stock_ticker
    
    for username in usernames:
        tweets = api.user_timeline(screen_name=username, count=100)
        for tweet in tweets:
            if stock_ticker in tweet.text or stock_ticker_full in tweet.text:
                data.append(tweet.text)

    tweets = api.search_tweets(q=stock_hashtag, count=100)
    for tweet in tweets:
        data.append(tweet.text)
        
    cleaned_data = []
    def generate_clean_ngrams(tweet_text, n):
        
        stop_words = list(stopwords.words('english')) + list(get_stop_words('en'))
        pattern = '[0-9]'
        
        tweet_text = tweet_text.lower()
        tweet_text = re.sub(r'[^a-zA-Z0-9\s]', ' ', tweet_text)
        tweet_text = re.sub(pattern, 'http', tweet_text)

        tokens = [token for token in tweet_text.split(' ') 
                  if token != '' 
                  and token not in stop_words 
                  and 'http' not in token]

        output = list(ngrams(tokens, n))
        return output
    
    for d in data:
        cleaned_data.append(generate_clean_ngrams(d, 2))

    analyzer = SentimentIntensityAnalyzer()

    score = []
    for data in cleaned_data:
        for d in data:
            bigram_text = " ".join(d)
            scores = analyzer.polarity_scores(bigram_text)
            score.append(scores['compound'])

    average_score = sum(score) / len(score)
    nseList['Sentiment'][nseList['Symbol'] == stock_ticker] = average_score
    #nseList['Sentiment'] = np.where(nseList['Symbol'] == stock_ticker,average_score,0)
    print('Index: ',index,' ',stock_ticker,': ',average_score)

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_26204\2694705966.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nseList['Sentiment'][nseList['Symbol'] == stock_ticker] = average_score


Index:  0   3MINDIA :  0.019135483870967734
Index:  1   ABB :  0.022205214285714297
Index:  2   ACC :  0.0576765065502183
Index:  3   AIAENG :  0.058044489795918364
Index:  4   APLAPOLLO :  0.04137260147601474
Index:  5   AUBANK :  0.0370515690376569
Index:  6   AARTIDRUGS :  -0.024600000000000018
Index:  7   AAVAS :  0.03237320261437907
Index:  8   ABBOTINDIA :  0.015310675381263622
Index:  9   ADANIENT :  0.04225381565906827
Index:  10   ADANIGREEN :  0.033898474059003085
Index:  11   ADANIPORTS :  0.02559132149901383
Index:  12   ATGL :  0.027436094674556214
Index:  13   ADANITRANS :  0.008110079575596809
Index:  14   AWL :  0.04625295095594346
Index:  15   ABCAPITAL :  0.028575646551724128
Index:  16   ABFRL :  0.03922133971291869
Index:  17   ABSLAMC :  0.03495357142857142
Index:  18   AEGISCHEM :  0.004453686635944701
Index:  19   AETHER :  0.03286536312849162
Index:  20   AFFLE :  0.01797924528301887
Index:  21   AJANTPHARM :  -0.03176744186046512
Index:  22   APLLTD :  -0.00885

ZeroDivisionError: division by zero

In [ ]:
nseList['Sentiment Group'] = np.where(nseList['Sentiment'] >= 0.05,'Postive Sentiment',nseList['Sentiment'] <= -0.05, 'Negative Sentiment','Neutral Sentiment')

In [ ]:
nseList.to_csv('C:/Users/abhit/Desktop/COLLEGE/SEM X/SEMX_Git/CC/submission.csv', index = False)